In [ ]:
from crossword_solver.candidate_search import *
from crossword_solver.crossword_utils import Crossword, load_crossword, Hint, Direction
from crossword_solver.solver_utils import solving_algorithm, display_results
import pandas as pd
from gensim.models import KeyedVectors

In [ ]:
from estnltk.taggers import SpellCheckRetagger
from estnltk import Text


In [ ]:
%timeit SpellCheckRetagger()

In [ ]:

%%timeit
text = Text('Ma tahax minna järve äärde ja püda hauge, katikaid ja karpe.').tag_layer(['words'])
spelling_tagger=SpellCheckRetagger()
spelling_tagger.retag(text)
orig_list = [w.text for w in text.words]
normalised = [w[0] for w in text.normalized_form]
clean_list = [normalised[i] if normalised[i] is not None else orig_list[i] for i in range(len(orig_list))]
clean = " ".join(clean_list)

In [ ]:
list(text.words)[0].normalized_form[0]

In [ ]:
%%timeit
text = Text('Ma tahax minna järve äärde ja püda hauge, katikaid ja karpe.').tag_layer(['words'])
spelling_tagger=SpellCheckRetagger()
spelling_tagger.retag(text)
clean_list = [span.normalized_form[0] if span.normalized_form[0] else span.text for span in text.words]
clean = " ".join(clean_list)
clean

In [ ]:
# Load word2vec
#word2vec_model = KeyedVectors.load_word2vec_format('../data/lemmas.cbow.s100.w2v.bin', binary=True)

In [ ]:
# Read in WordNet contents
wn_voc = pd.read_parquet('../data/wordnet_contents.parquet')

In [ ]:
# Read in crossword
cw = load_crossword('../data/crossword1.pickle')

In [ ]:
def search_candidates(hint):
    hint_variations = [hint.hint] # Later each hint should have a list inside already 
    length = hint.length
    candidate_list = []
    for text in hint_variations:
        lemmatised_text = lemmatise_text(text)
        candidate_list.extend(wikipedia_search(lemmatised_text))
        candidate_list.extend(wordnet_search(lemmatised_text))
        candidate_list.extend(create_abbreviated_candidate(text, length))
        candidate_list.extend(create_word2vec_candidates(lemmatised_text))
    clean_candidates(candidate_list)
    candidate_list = remove_duplicates(candidate_list)
    candidate_list = expand_candidates(candidate_list)
    candidate_list = remove_duplicates(candidate_list)
    sorted_candidates = sorted(candidate_list, key = lambda x: x.weight, reverse = True)
    return sorted_candidates

In [ ]:
hint = cw.hints[1]
hint

In [ ]:
search_candidates(hint)

In [ ]:
WORDNET_VOCAB = 'wordnet vocab'
WORDNET_VOCAB_WEIGHT = 0.8
def find_regex_match(hint_text, length, wn_list):
    if "..." not in hint_text:
        return []
    matching_group = f"(.{{{length}}})"
    pattern ="^"+ hint_text.replace("...", matching_group)+ "$"
    matches = []
    for w in wn_list:
        match = re.search(pattern, w)
        if match is not None:
            matches.append(match.group(1))
    if len(matches) > 0:
        return convert_results_to_candidates(matches, tag = WORDNET_VOCAB, weight = WORDNET_VOCAB_WEIGHT)
    return []

In [ ]:
hint.hint = 'kilo...'
hint.length = 5
find_regex_match(hint.hint, hint.length, wn_voc['word'])